# PROJECT
Project creating probabilities of households picking each choice of the 500 alternatives from body type, fuel type and age

In [3]:
import pandas as pd
import numpy as np
import math

In [ ]:
'''
these files will be our input and contain household data from NHTS, vehicle cost data
and EPA data on body type, fuel type and age

'''
epa_and_vehiclecost   = pd.read_csv('../inputs/combined epa and veh cost.csv')
epa_and_vehiclecost .set_index("CHOICE4",inplace=True,drop=True)
NHTS = pd.read_csv('../inputs/NHTS2017.csv')
NHTS.set_index("HOUSEID",inplace=True,drop=True)


In [ ]:
vehf12  = pd.read_csv('../inputs/vehtypef12.csv', usecols= [2,4])
vehf12.dropna(subset=["variable"],inplace=True)
vehf12.set_index("variable",inplace=True,drop=True)

In [ ]:
'''
These variables are defined in the alogit file and will be making use of codes from the NHTS codebook and table
   
    INC025   = ifin(HHFAMINC,1,3)      
    INC2550  = ifin(HHFAMINC,4,5)      
    INC50100 = ifin(HHFAMINC,6,7)   
    INC100150= ifin(HHFAMINC,8,9)  
    INC150P  = ifge(HHFAMINC,10)  
    INC_MISS = iflt(HHFAMINC,0)
    RURALa(a) = a * RURAL
    ONEVEH = ifeq(HHVEHCNT,1)
    VHLTDR = ifgt(HHVEHCNT,1) * iflt(HHVEHCNT,PERSONS16P)
    VHEQDR = ifgt(HHVEHCNT,1) * ifeq(HHVEHCNT,PERSONS16P)
    VHGTDR = ifgt(HHVEHCNT,1) * ifgt(HHVEHCNT,PERSONS16P)
    NUM_CHLD3 = min(YOUNGCHILD + CHILDREN, 3)
    DSTWK = min(DSTTOWKAVG, 100)
    SFO = ifeq(HH_CBSA, 41860) + ifeq(HH_CBSA, 47900)
    SAN = ifeq(HH_CBSA, 41740)
    ATL = ifeq(HH_CBSA, 12060) 
    SEA = ifeq(HH_CBSA, 42660) 
    DET = ifeq(HH_CBSA, 19820) 
    MSP = ifeq(HH_CBSA, 33460) 
    DCA = ifeq(HH_CBSA, 47900) 
    OREG = ifeq(HHSTFIPS, 41) 
    OHIO = ifeq(HHSTFIPS, 39)  
    SMSA = ifeq(HH_CBSA, 0) * ifeq(RURAL, 0)
    OWN_VAN = ifge(VAN_COUNT, 1)
    OWN_SUV = ifge(SUV_COUNT, 1)
    OWN_PU  = ifge(PU_COUNT, 1)
    OWN_MC  = ifge(MC_COUNT, 1)
    OWN_HYB = ifge(HYB_COUNT, 1)
    OWN_PEV = ifge(PEV_COUNT, 1)
    OWN_BEV = ifge(BEV_COUNT, 1)
    OWN_EV = ifge(PEV_COUNT + BEV_COUNT, 1)
    DEN1 = ifeq(HBRESDN, 50)
    DEN2 = ifeq(HBRESDN, 300)
    DEN3 = ifeq(HBRESDN, 750)
    DEN4 = ifeq(HBRESDN, 1500)
    DEN5 = ifeq(HBRESDN, 3000)
    DEN6 = ifeq(HBRESDN, 7000)
    DEN7 = ifeq(HBRESDN, 17000)
    DEN8 = ifeq(HBRESDN, 30000)
    OWN_VAN = ifge(VAN_COUNT, 1)
    OWN_SUV = ifge(SUV_COUNT, 1)
    OWN_PU  = ifge(PU_COUNT, 1)
    OWN_MC  = ifge(MC_COUNT, 1)
    OWN_HYB = ifge(HYB_COUNT, 1)
    OWN_PEV = ifge(PEV_COUNT, 1)
    OWN_BEV = ifge(BEV_COUNT, 1)
    OWN_EV = ifge(PEV_COUNT + BEV_COUNT, 1)
    DEN1 = ifeq(HBRESDN, 50)
    DEN2 = ifeq(HBRESDN, 300)
    DEN3 = ifeq(HBRESDN, 750)
    DEN4 = ifeq(HBRESDN, 1500)
    DEN5 = ifeq(HBRESDN, 3000)
    DEN6 = ifeq(HBRESDN, 7000)
    DEN7 = ifeq(HBRESDN, 17000)
    DEN8 = ifeq(HBRESDN, 30000)
    RURAL = ifeq(URBRUR,2
'''

In [5]:
#starting with income which is the first 6 variables above
#defining the columns that give the conditions for HHFAMINC
NHTS['INC025'] = 0
NHTS['INC025'] = np.where((NHTS['HHFAMINC']>=1) & (NHTS['HHFAMINC']<=3), 1, 0)
NHTS['INC2550'] = 0
NHTS['INC2550'] = np.where((NHTS['HHFAMINC']>=4) & (NHTS['HHFAMINC']<=5), 1, 0)
NHTS['INC50100'] = 0
NHTS['INC50100'] = np.where((NHTS['HHFAMINC']>=6) & (NHTS['HHFAMINC']<=7), 1, 0)
NHTS['INC100150'] = 0
NHTS['INC100150'] = np.where((NHTS['HHFAMINC']>=8) & (NHTS['HHFAMINC']<=9), 1, 0)
NHTS['INC150P'] = 0
NHTS['INC150P'] = np.where((NHTS['HHFAMINC']>=10), 1, 0)
NHTS['INC_MISS'] = 0
NHTS['INC_MISS'] = np.where((NHTS['HHFAMINC']<=0), 1, 0)
NHTS['ONEVEH'] = 0
NHTS['ONEVEH'] = np.where((NHTS['HHVEHCNT'] ==1), 1, 0)
NHTS['VHLTDR'] = 0
NHTS['VHLTDR'] = np.where((NHTS['HHVEHCNT']>1) & (NHTS['HHVEHCNT'] < NHTS['PERSONS16P']), 1, 0)
NHTS['VHEQDR'] = 0
NHTS['VHEQDR'] = np.where((NHTS['HHVEHCNT']>1) & (NHTS['HHVEHCNT'] == NHTS['PERSONS16P']), 1, 0)
NHTS['VHGTDR'] = 0
NHTS['VHGTDR'] = np.where((NHTS['HHVEHCNT']>1) & (NHTS['HHVEHCNT'] > NHTS['PERSONS16P']), 1, 0)
NHTS['NUM_CHILD3'] = 0
NHTS['NUM_CHILD3'] = np.where((min(NHTS['YOUNGCHILD'] + NHTS['CHILDREN']) == 3), 1, 0)
NHTS['DSTWK'] = 0
NHTS['DSTWK'] = np.where(min(NHTS['DISTTOWK_AVG'] == 100), 1, 0)
NHTS['SFO'] = 0
NHTS['SFO'] = np.where((NHTS['HH_CBSA'] == 41860) & (NHTS['HH_CBSA'] == 47900), 1, 0)
NHTS['SAN'] = 0
NHTS['SAN'] = np.where((NHTS['HH_CBSA'] == 41740), 1, 0)
NHTS['ATL'] = 0
NHTS['ATL'] = np.where((NHTS['HH_CBSA'] == 12060), 1, 0)
NHTS['SEA'] = 0
NHTS['SEA'] = np.where((NHTS['HH_CBSA'] == 42660), 1, 0)
NHTS['DET'] = 0
NHTS['DET'] = np.where((NHTS['HH_CBSA'] == 19820), 1, 0)
NHTS['MSP'] = 0
NHTS['MSP'] = np.where((NHTS['HH_CBSA'] == 33460), 1, 0)
NHTS['DCA'] = 0
NHTS['DCA'] = np.where((NHTS['HH_CBSA'] == 47900), 1, 0)
NHTS['OREG'] = 0
NHTS['OREG'] = np.where((NHTS['HHSTFIPS'] == 41), 1, 0)
NHTS['OHIO'] = 0
NHTS['OHIO'] = np.where((NHTS['HHSTFIPS'] == 39), 1, 0)
NHTS['OWN_VAN'] = 0
NHTS['OWN_VAN'] = np.where((NHTS['VAN_COUNT'] >= 1), 1, 0)
NHTS['OWN_PU'] = 0
NHTS['OWN_PU'] = np.where((NHTS['PU_COUNT'] >= 1), 1, 0)
NHTS['OWN_SUV'] = 0
NHTS['OWN_SUV'] = np.where((NHTS['SUV_COUNT'] >= 1), 1, 0)
NHTS['OWN_MC'] = 0
NHTS['OWN_MC'] = np.where((NHTS['MC_COUNT'] >= 1), 1, 0)
NHTS['OWN_HYB'] = 0
NHTS['OWN_HYB'] = np.where((NHTS['HYB_COUNT'] >= 1), 1, 0)
NHTS['OWN_PEV'] = 0
NHTS['OWN_PEV'] = np.where((NHTS['PEV_COUNT'] >= 1), 1, 0)
NHTS['OWN_BEV'] = 0
NHTS['OWN_BEV'] = np.where((NHTS['BEV_COUNT'] >= 1), 1, 0)
NHTS['OWN_EV'] = 0
NHTS['OWN_EV'] = np.where((NHTS['PEV_COUNT'] + NHTS['BEV_COUNT'] >= 1), 1, 0)
NHTS['DEN1'] = 0
NHTS['DEN1'] = np.where((NHTS['HBRESDN'] == 50), 1, 0)
NHTS['DEN2'] = 0
NHTS['DEN2'] = np.where((NHTS['HBRESDN'] == 300), 1, 0)
NHTS['DEN3'] = 0
NHTS['DEN3'] = np.where((NHTS['HBRESDN'] == 750), 1, 0)
NHTS['DEN4'] = 0
NHTS['DEN4'] = np.where((NHTS['HBRESDN'] == 1500), 1, 0)
NHTS['DEN5'] = 0
NHTS['DEN5'] = np.where((NHTS['HBRESDN'] == 3000), 1, 0)
NHTS['DEN6'] = 0
NHTS['DEN6'] = np.where((NHTS['HBRESDN'] == 7000), 1, 0)
NHTS['DEN7'] = 0
NHTS['DE71'] = np.where((NHTS['HBRESDN'] == 17000), 1, 0)
NHTS['DEN8'] = 0
NHTS['DEN8'] = np.where((NHTS['HBRESDN'] == 30000), 1, 0)
NHTS['ALL_VAN'] = 0
NHTS['ALL_VAN'] = np.where((NHTS['VAN_COUNT'] == NHTS['HHVEHCNT']), 1, 0)
NHTS['ALL_SUV'] = 0
NHTS['ALL_SUV'] = np.where((NHTS['SUV_COUNT'] == NHTS['HHVEHCNT']), 1, 0)
NHTS['ALL_PU'] = 0
NHTS['ALL_PU'] = np.where((NHTS['PU_COUNT'] == NHTS['HHVEHCNT']), 1, 0)
NHTS['ALL_MC'] = 0
NHTS['ALL_MC'] = np.where((NHTS['MC_COUNT'] == NHTS['HHVEHCNT']), 1, 0)
NHTS['ALL_HYB'] = 0
NHTS['ALL_HYB'] = np.where((NHTS['HYB_COUNT'] == NHTS['HHVEHCNT']), 1, 0)
NHTS['ALL_PEV'] = 0
NHTS['ALL_PEV'] = np.where((NHTS['PEV_COUNT'] == NHTS['HHVEHCNT']), 1, 0)
NHTS['ALL_BEV'] = 0
NHTS['ALL_BEV'] = np.where((NHTS['BEV_COUNT'] == NHTS['HHVEHCNT']), 1, 0)

In [6]:
NHTS

,Unnamed: 0,VEHID,RANK,VEHYEAR,VEHAGE,MAKE,MODEL,FUELTYPE,VEHTYPE,WHOMAIN,...,DEN7,DE71,DEN8,ALL_VAN,ALL_SUV,ALL_PU,ALL_MC,ALL_HYB,ALL_PEV,ALL_BEV
HOUSEID,,,,,,,,,,,,,,,,,,,,,
30000007,0,1,1,2007,10,49,49032,1,1,3,...,0,0,0,0,0,0,0,0,0,0
30000007,1,2,4,2004,13,49,49442,1,2,-8,...,0,0,0,0,0,0,0,0,0,0
30000007,2,3,3,1998,19,19,19014,1,1,1,...,0,0,0,0,0,0,0,0,0,0
30000007,3,5,2,1993,24,20,20481,1,4,2,...,0,0,0,0,0,0,0,0,0,0
30000008,4,1,4,2014,3,20,20028,1,1,97,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40794294,247684,2,1,2011,6,20,20026,3,1,1,...,0,0,0,0,0,0,0,0,0,0
40794301,247685,1,1,2012,5,37,37441,1,2,1,...,0,0,0,0,0,0,0,0,0,0
40794301,247686,2,2,2005,12,49,49032,1,1,4,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
'''
Vehicle type: car:1, van:2, suv:3, pickup:4, motorcycle:5
Age: 1 - 20 years
Fuel type: gas:1, diesel:2, hybrid:3, pev:4, bev:5


veh_type = ["1","2","3","4","5"]
fuel_type = ["1","2","3","4","5"]
car_age = ["01","02","03","04","05","06","07","08","09","10","11","12","13","14","15","16","17","18","19","20"]
for veh in veh_type:
    for fuel in fuel_type:
        for age in car_age:
            temp = veh + fuel + str(age)
            print(f"Choice: {temp}")
            
    
 '''   

In [8]:
df = pd.DataFrame()
df

""


In [ ]:
'''
define a method where we find the utility for each household picking the choice
This code is the calculation for picking choices 101 - 120: cars ages 1 - 20
The results of this is the utility for each household for each choice

'''

df["util_101"] = vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO']                                    
df["util_102"] = vehf12.loc['age2', 'Coefficient'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO']
df["util_103"] = vehf12.loc['age3', 'Coefficient'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO']
df["util_104"] = vehf12.loc['age4', 'Coefficient'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO']
df["util_105"] = vehf12.loc['age5', 'Coefficient'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO']
df["util_106"] = vehf12.loc['age6', 'Coefficient'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO']
df["util_107"] = vehf12.loc['age7', 'Coefficient'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO']
df["util_108"] = vehf12.loc['age8', 'Coefficient'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO']
df["util_109"] = vehf12.loc['age9', 'Coefficient'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO']
df["util_110"] = vehf12.loc['age10', 'Coefficient'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO']
df["util_111"] = vehf12.loc['age11', 'Coefficient'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO']
df["util_112"] = vehf12.loc['age12', 'Coefficient'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO']
df["util_113"] = vehf12.loc['age13', 'Coefficient'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO']
df["util_114"] = vehf12.loc['age14', 'Coefficient'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO']
df["util_115"] = vehf12.loc['age15', 'Coefficient'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO']
df["util_116"] = vehf12.loc['age16', 'Coefficient'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO']
df["util_117"] = vehf12.loc['age17', 'Coefficient'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO']
df["util_118"] = vehf12.loc['age18', 'Coefficient'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO']
df["util_119"] = vehf12.loc['age19', 'Coefficient'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO']
df["util_120"] = vehf12.loc['age20', 'Coefficient'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO']



In [10]:
#van aged 1 -20 years
df["util_201"] = vehf12.loc['van', 'Coefficient'] + vehf12.loc['i025_van', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_van', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_van', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_van', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_van', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_van', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_van', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_van', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_van', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_van', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_van', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_van', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_van', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_van', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_van', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_van', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_van', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_van', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_van', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_van', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_van', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_van', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_van', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['van_suv', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['van_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['van_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_van', 'Coefficient'] * NHTS['ALL_VAN']
df["util_202"] = vehf12.loc['van', 'Coefficient'] + vehf12.loc['age2', 'Coefficient'] + vehf12.loc['i025_van', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_van', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_van', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_van', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_van', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_van', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_van', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_van', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_van', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_van', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_van', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_van', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_van', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_van', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_van', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_van', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_van', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_van', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_van', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_van', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_van', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_van', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_van', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['van_suv', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['van_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['van_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_van', 'Coefficient'] * NHTS['ALL_VAN']
df["util_203"] = vehf12.loc['van', 'Coefficient'] + vehf12.loc['age3', 'Coefficient'] + vehf12.loc['i025_van', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_van', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_van', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_van', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_van', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_van', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_van', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_van', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_van', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_van', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_van', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_van', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_van', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_van', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_van', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_van', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_van', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_van', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_van', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_van', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_van', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_van', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_van', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['van_suv', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['van_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['van_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_van', 'Coefficient'] * NHTS['ALL_VAN']
df["util_204"] = vehf12.loc['van', 'Coefficient'] + vehf12.loc['age4', 'Coefficient'] + vehf12.loc['i025_van', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_van', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_van', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_van', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_van', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_van', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_van', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_van', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_van', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_van', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_van', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_van', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_van', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_van', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_van', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_van', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_van', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_van', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_van', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_van', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_van', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_van', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_van', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['van_suv', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['van_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['van_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_van', 'Coefficient'] * NHTS['ALL_VAN']
df["util_205"] = vehf12.loc['van', 'Coefficient'] + vehf12.loc['age5', 'Coefficient'] + vehf12.loc['i025_van', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_van', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_van', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_van', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_van', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_van', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_van', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_van', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_van', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_van', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_van', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_van', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_van', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_van', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_van', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_van', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_van', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_van', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_van', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_van', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_van', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_van', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_van', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['van_suv', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['van_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['van_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_van', 'Coefficient'] * NHTS['ALL_VAN']
df["util_206"] = vehf12.loc['van', 'Coefficient'] + vehf12.loc['age6', 'Coefficient'] + vehf12.loc['i025_van', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_van', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_van', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_van', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_van', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_van', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_van', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_van', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_van', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_van', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_van', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_van', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_van', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_van', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_van', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_van', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_van', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_van', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_van', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_van', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_van', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_van', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_van', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['van_suv', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['van_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['van_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_van', 'Coefficient'] * NHTS['ALL_VAN']
df["util_207"] = vehf12.loc['van', 'Coefficient'] + vehf12.loc['age7', 'Coefficient'] + vehf12.loc['i025_van', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_van', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_van', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_van', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_van', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_van', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_van', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_van', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_van', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_van', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_van', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_van', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_van', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_van', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_van', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_van', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_van', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_van', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_van', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_van', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_van', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_van', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_van', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['van_suv', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['van_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['van_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_van', 'Coefficient'] * NHTS['ALL_VAN']
df["util_208"] = vehf12.loc['van', 'Coefficient'] + vehf12.loc['age8', 'Coefficient'] + vehf12.loc['i025_van', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_van', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_van', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_van', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_van', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_van', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_van', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_van', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_van', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_van', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_van', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_van', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_van', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_van', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_van', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_van', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_van', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_van', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_van', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_van', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_van', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_van', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_van', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['van_suv', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['van_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['van_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_van', 'Coefficient'] * NHTS['ALL_VAN']
df["util_209"] = vehf12.loc['van', 'Coefficient'] + vehf12.loc['age9', 'Coefficient'] + vehf12.loc['i025_van', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_van', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_van', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_van', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_van', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_van', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_van', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_van', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_van', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_van', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_van', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_van', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_van', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_van', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_van', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_van', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_van', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_van', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_van', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_van', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_van', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_van', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_van', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['van_suv', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['van_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['van_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_van', 'Coefficient'] * NHTS['ALL_VAN']
df["util_210"] = vehf12.loc['van', 'Coefficient'] + vehf12.loc['age10', 'Coefficient'] + vehf12.loc['i025_van', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_van', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_van', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_van', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_van', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_van', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_van', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_van', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_van', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_van', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_van', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_van', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_van', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_van', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_van', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_van', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_van', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_van', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_van', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_van', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_van', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_van', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_van', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['van_suv', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['van_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['van_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_van', 'Coefficient'] * NHTS['ALL_VAN']
df["util_211"] = vehf12.loc['van', 'Coefficient'] + vehf12.loc['age11', 'Coefficient'] + vehf12.loc['i025_van', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_van', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_van', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_van', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_van', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_van', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_van', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_van', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_van', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_van', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_van', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_van', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_van', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_van', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_van', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_van', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_van', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_van', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_van', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_van', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_van', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_van', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_van', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['van_suv', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['van_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['van_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_van', 'Coefficient'] * NHTS['ALL_VAN']
df["util_212"] = vehf12.loc['van', 'Coefficient'] + vehf12.loc['age12', 'Coefficient'] + vehf12.loc['i025_van', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_van', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_van', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_van', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_van', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_van', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_van', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_van', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_van', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_van', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_van', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_van', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_van', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_van', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_van', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_van', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_van', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_van', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_van', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_van', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_van', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_van', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_van', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['van_suv', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['van_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['van_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_van', 'Coefficient'] * NHTS['ALL_VAN']
df["util_213"] = vehf12.loc['van', 'Coefficient'] + vehf12.loc['age13', 'Coefficient'] + vehf12.loc['i025_van', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_van', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_van', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_van', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_van', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_van', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_van', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_van', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_van', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_van', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_van', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_van', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_van', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_van', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_van', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_van', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_van', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_van', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_van', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_van', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_van', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_van', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_van', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['van_suv', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['van_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['van_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_van', 'Coefficient'] * NHTS['ALL_VAN']
df["util_214"] = vehf12.loc['van', 'Coefficient'] + vehf12.loc['age14', 'Coefficient'] + vehf12.loc['i025_van', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_van', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_van', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_van', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_van', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_van', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_van', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_van', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_van', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_van', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_van', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_van', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_van', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_van', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_van', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_van', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_van', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_van', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_van', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_van', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_van', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_van', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_van', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['van_suv', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['van_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['van_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_van', 'Coefficient'] * NHTS['ALL_VAN']
df["util_215"] = vehf12.loc['van', 'Coefficient'] + vehf12.loc['age15', 'Coefficient'] + vehf12.loc['i025_van', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_van', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_van', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_van', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_van', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_van', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_van', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_van', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_van', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_van', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_van', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_van', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_van', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_van', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_van', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_van', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_van', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_van', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_van', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_van', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_van', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_van', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_van', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['van_suv', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['van_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['van_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_van', 'Coefficient'] * NHTS['ALL_VAN']
df["util_216"] = vehf12.loc['van', 'Coefficient'] + vehf12.loc['age16', 'Coefficient'] + vehf12.loc['i025_van', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_van', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_van', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_van', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_van', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_van', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_van', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_van', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_van', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_van', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_van', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_van', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_van', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_van', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_van', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_van', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_van', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_van', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_van', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_van', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_van', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_van', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_van', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['van_suv', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['van_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['van_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_van', 'Coefficient'] * NHTS['ALL_VAN']
df["util_217"] = vehf12.loc['van', 'Coefficient'] + vehf12.loc['age17', 'Coefficient'] + vehf12.loc['i025_van', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_van', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_van', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_van', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_van', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_van', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_van', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_van', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_van', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_van', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_van', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_van', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_van', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_van', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_van', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_van', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_van', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_van', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_van', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_van', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_van', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_van', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_van', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['van_suv', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['van_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['van_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_van', 'Coefficient'] * NHTS['ALL_VAN']
df["util_218"] = vehf12.loc['van', 'Coefficient'] + vehf12.loc['age18', 'Coefficient'] + vehf12.loc['i025_van', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_van', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_van', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_van', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_van', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_van', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_van', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_van', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_van', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_van', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_van', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_van', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_van', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_van', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_van', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_van', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_van', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_van', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_van', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_van', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_van', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_van', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_van', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['van_suv', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['van_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['van_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_van', 'Coefficient'] * NHTS['ALL_VAN']
df["util_219"] = vehf12.loc['van', 'Coefficient'] + vehf12.loc['age19', 'Coefficient'] + vehf12.loc['i025_van', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_van', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_van', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_van', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_van', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_van', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_van', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_van', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_van', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_van', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_van', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_van', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_van', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_van', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_van', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_van', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_van', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_van', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_van', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_van', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_van', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_van', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_van', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['van_suv', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['van_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['van_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_van', 'Coefficient'] * NHTS['ALL_VAN']
df["util_220"] = vehf12.loc['van', 'Coefficient'] + vehf12.loc['age20', 'Coefficient'] + vehf12.loc['i025_van', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_van', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_van', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_van', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_van', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_van', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_van', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_van', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_van', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_van', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_van', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_van', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_van', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_van', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_van', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_van', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_van', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_van', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_van', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_van', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_van', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_van', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_van', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['van_suv', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['van_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['van_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_van', 'Coefficient'] * NHTS['ALL_VAN']


In [11]:
# suv aged 1 -20 years
df["util_301"] = vehf12.loc['suv', 'Coefficient'] + vehf12.loc['i025_suv', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_suv', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_suv', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_suv', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_suv', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_suv', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_suv', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_suv', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_suv', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_suv', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_suv', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_suv', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_suv', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_suv', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_suv', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_suv', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_suv', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_suv', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_suv', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_suv', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_suv', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_suv', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_suv', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_suv', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['suv_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['suv_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_suv', 'Coefficient'] * NHTS['ALL_SUV']
df["util_302"] = vehf12.loc['suv', 'Coefficient'] + vehf12.loc['age2', 'Coefficient'] + vehf12.loc['i025_suv', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_suv', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_suv', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_suv', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_suv', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_suv', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_suv', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_suv', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_suv', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_suv', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_suv', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_suv', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_suv', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_suv', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_suv', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_suv', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_suv', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_suv', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_suv', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_suv', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_suv', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_suv', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_suv', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_suv', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['suv_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['suv_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_suv', 'Coefficient'] * NHTS['ALL_SUV']
df["util_303"] = vehf12.loc['suv', 'Coefficient'] + vehf12.loc['age3', 'Coefficient'] + vehf12.loc['i025_suv', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_suv', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_suv', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_suv', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_suv', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_suv', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_suv', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_suv', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_suv', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_suv', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_suv', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_suv', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_suv', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_suv', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_suv', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_suv', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_suv', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_suv', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_suv', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_suv', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_suv', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_suv', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_suv', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_suv', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['suv_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['suv_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_suv', 'Coefficient'] * NHTS['ALL_SUV']
df["util_304"] = vehf12.loc['suv', 'Coefficient'] + vehf12.loc['age4', 'Coefficient'] + vehf12.loc['i025_suv', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_suv', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_suv', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_suv', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_suv', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_suv', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_suv', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_suv', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_suv', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_suv', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_suv', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_suv', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_suv', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_suv', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_suv', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_suv', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_suv', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_suv', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_suv', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_suv', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_suv', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_suv', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_suv', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_suv', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['suv_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['suv_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_suv', 'Coefficient'] * NHTS['ALL_SUV'] 
df["util_305"] = vehf12.loc['suv', 'Coefficient'] + vehf12.loc['age5', 'Coefficient'] + vehf12.loc['i025_suv', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_suv', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_suv', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_suv', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_suv', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_suv', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_suv', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_suv', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_suv', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_suv', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_suv', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_suv', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_suv', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_suv', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_suv', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_suv', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_suv', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_suv', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_suv', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_suv', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_suv', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_suv', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_suv', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_suv', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['suv_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['suv_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_suv', 'Coefficient'] * NHTS['ALL_SUV'] 
df["util_306"] = vehf12.loc['suv', 'Coefficient'] + vehf12.loc['age6', 'Coefficient'] + vehf12.loc['i025_suv', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_suv', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_suv', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_suv', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_suv', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_suv', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_suv', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_suv', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_suv', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_suv', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_suv', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_suv', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_suv', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_suv', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_suv', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_suv', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_suv', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_suv', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_suv', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_suv', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_suv', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_suv', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_suv', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_suv', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['suv_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['suv_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_suv', 'Coefficient'] * NHTS['ALL_SUV'] 
df["util_307"] = vehf12.loc['suv', 'Coefficient'] + vehf12.loc['age7', 'Coefficient'] + vehf12.loc['i025_suv', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_suv', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_suv', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_suv', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_suv', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_suv', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_suv', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_suv', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_suv', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_suv', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_suv', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_suv', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_suv', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_suv', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_suv', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_suv', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_suv', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_suv', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_suv', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_suv', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_suv', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_suv', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_suv', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_suv', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['suv_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['suv_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_suv', 'Coefficient'] * NHTS['ALL_SUV'] 
df["util_308"] = vehf12.loc['suv', 'Coefficient'] + vehf12.loc['age8', 'Coefficient'] + vehf12.loc['i025_suv', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_suv', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_suv', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_suv', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_suv', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_suv', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_suv', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_suv', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_suv', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_suv', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_suv', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_suv', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_suv', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_suv', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_suv', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_suv', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_suv', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_suv', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_suv', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_suv', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_suv', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_suv', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_suv', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_suv', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['suv_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['suv_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_suv', 'Coefficient'] * NHTS['ALL_SUV'] 
df["util_309"] = vehf12.loc['suv', 'Coefficient'] + vehf12.loc['age9', 'Coefficient'] + vehf12.loc['i025_suv', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_suv', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_suv', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_suv', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_suv', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_suv', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_suv', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_suv', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_suv', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_suv', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_suv', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_suv', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_suv', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_suv', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_suv', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_suv', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_suv', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_suv', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_suv', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_suv', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_suv', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_suv', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_suv', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_suv', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['suv_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['suv_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_suv', 'Coefficient'] * NHTS['ALL_SUV'] 
df["util_310"] = vehf12.loc['suv', 'Coefficient'] + vehf12.loc['age10', 'Coefficient'] + vehf12.loc['i025_suv', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_suv', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_suv', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_suv', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_suv', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_suv', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_suv', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_suv', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_suv', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_suv', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_suv', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_suv', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_suv', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_suv', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_suv', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_suv', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_suv', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_suv', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_suv', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_suv', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_suv', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_suv', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_suv', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_suv', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['suv_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['suv_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_suv', 'Coefficient'] * NHTS['ALL_SUV'] 
df["util_311"] = vehf12.loc['suv', 'Coefficient'] + vehf12.loc['age11', 'Coefficient'] + vehf12.loc['i025_suv', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_suv', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_suv', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_suv', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_suv', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_suv', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_suv', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_suv', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_suv', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_suv', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_suv', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_suv', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_suv', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_suv', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_suv', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_suv', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_suv', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_suv', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_suv', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_suv', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_suv', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_suv', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_suv', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_suv', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['suv_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['suv_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_suv', 'Coefficient'] * NHTS['ALL_SUV'] 
df["util_312"] = vehf12.loc['suv', 'Coefficient'] + vehf12.loc['age12', 'Coefficient'] + vehf12.loc['i025_suv', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_suv', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_suv', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_suv', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_suv', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_suv', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_suv', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_suv', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_suv', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_suv', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_suv', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_suv', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_suv', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_suv', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_suv', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_suv', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_suv', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_suv', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_suv', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_suv', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_suv', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_suv', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_suv', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_suv', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['suv_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['suv_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_suv', 'Coefficient'] * NHTS['ALL_SUV'] 
df["util_313"] = vehf12.loc['suv', 'Coefficient'] + vehf12.loc['age13', 'Coefficient'] + vehf12.loc['i025_suv', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_suv', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_suv', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_suv', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_suv', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_suv', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_suv', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_suv', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_suv', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_suv', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_suv', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_suv', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_suv', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_suv', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_suv', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_suv', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_suv', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_suv', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_suv', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_suv', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_suv', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_suv', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_suv', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_suv', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['suv_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['suv_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_suv', 'Coefficient'] * NHTS['ALL_SUV'] 
df["util_314"] = vehf12.loc['suv', 'Coefficient'] + vehf12.loc['age14', 'Coefficient'] + vehf12.loc['i025_suv', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_suv', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_suv', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_suv', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_suv', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_suv', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_suv', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_suv', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_suv', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_suv', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_suv', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_suv', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_suv', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_suv', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_suv', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_suv', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_suv', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_suv', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_suv', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_suv', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_suv', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_suv', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_suv', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_suv', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['suv_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['suv_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_suv', 'Coefficient'] * NHTS['ALL_SUV'] 
df["util_315"] = vehf12.loc['suv', 'Coefficient'] + vehf12.loc['age15', 'Coefficient'] + vehf12.loc['i025_suv', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_suv', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_suv', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_suv', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_suv', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_suv', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_suv', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_suv', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_suv', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_suv', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_suv', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_suv', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_suv', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_suv', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_suv', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_suv', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_suv', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_suv', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_suv', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_suv', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_suv', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_suv', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_suv', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_suv', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['suv_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['suv_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_suv', 'Coefficient'] * NHTS['ALL_SUV'] 
df["util_316"] = vehf12.loc['suv', 'Coefficient'] + vehf12.loc['age16', 'Coefficient'] + vehf12.loc['i025_suv', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_suv', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_suv', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_suv', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_suv', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_suv', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_suv', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_suv', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_suv', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_suv', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_suv', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_suv', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_suv', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_suv', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_suv', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_suv', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_suv', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_suv', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_suv', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_suv', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_suv', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_suv', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_suv', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_suv', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['suv_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['suv_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_suv', 'Coefficient'] * NHTS['ALL_SUV'] 
df["util_317"] = vehf12.loc['suv', 'Coefficient'] + vehf12.loc['age17', 'Coefficient'] + vehf12.loc['i025_suv', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_suv', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_suv', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_suv', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_suv', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_suv', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_suv', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_suv', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_suv', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_suv', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_suv', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_suv', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_suv', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_suv', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_suv', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_suv', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_suv', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_suv', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_suv', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_suv', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_suv', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_suv', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_suv', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_suv', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['suv_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['suv_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_suv', 'Coefficient'] * NHTS['ALL_SUV'] 
df["util_318"] = vehf12.loc['suv', 'Coefficient'] + vehf12.loc['age18', 'Coefficient'] + vehf12.loc['i025_suv', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_suv', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_suv', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_suv', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_suv', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_suv', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_suv', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_suv', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_suv', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_suv', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_suv', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_suv', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_suv', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_suv', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_suv', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_suv', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_suv', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_suv', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_suv', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_suv', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_suv', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_suv', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_suv', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_suv', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['suv_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['suv_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_suv', 'Coefficient'] * NHTS['ALL_SUV'] 
df["util_319"] = vehf12.loc['suv', 'Coefficient'] + vehf12.loc['age19', 'Coefficient'] + vehf12.loc['i025_suv', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_suv', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_suv', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_suv', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_suv', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_suv', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_suv', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_suv', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_suv', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_suv', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_suv', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_suv', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_suv', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_suv', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_suv', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_suv', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_suv', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_suv', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_suv', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_suv', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_suv', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_suv', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_suv', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_suv', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['suv_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['suv_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_suv', 'Coefficient'] * NHTS['ALL_SUV'] 
df["util_320"] = vehf12.loc['suv', 'Coefficient'] + vehf12.loc['age20', 'Coefficient'] + vehf12.loc['i025_suv', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_suv', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_suv', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_suv', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_suv', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_suv', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_suv', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_suv', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_suv', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_suv', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_suv', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_suv', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_suv', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_suv', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_suv', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_suv', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_suv', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_suv', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_suv', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_suv', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_suv', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_suv', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_suv', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_suv', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['suv_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['suv_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_suv', 'Coefficient'] * NHTS['ALL_SUV'] 


In [12]:
#pickup aged 1-20 years
df["util_401"] = vehf12.loc['pu', 'Coefficient'] + vehf12.loc['i025_pu', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_pu', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_pu', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_pu', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_pu', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_pu', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_pu', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_pu', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_pu', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_pu', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_pu', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_pu', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_pu', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_pu', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_pu', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_pu', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_pu', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_pu', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_pu', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_pu', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_pu', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_pu', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_pu', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_pu', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['pu_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['pu_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_pu', 'Coefficient'] * NHTS['ALL_PU']
df["util_402"] = vehf12.loc['pu', 'Coefficient'] + vehf12.loc['age2', 'Coefficient'] + vehf12.loc['i025_pu', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_pu', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_pu', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_pu', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_pu', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_pu', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_pu', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_pu', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_pu', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_pu', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_pu', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_pu', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_pu', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_pu', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_pu', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_pu', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_pu', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_pu', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_pu', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_pu', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_pu', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_pu', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_pu', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_pu', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['pu_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['pu_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_pu', 'Coefficient'] * NHTS['ALL_PU']
df["util_403"] = vehf12.loc['pu', 'Coefficient'] + vehf12.loc['age3', 'Coefficient'] + vehf12.loc['i025_pu', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_pu', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_pu', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_pu', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_pu', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_pu', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_pu', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_pu', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_pu', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_pu', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_pu', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_pu', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_pu', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_pu', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_pu', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_pu', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_pu', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_pu', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_pu', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_pu', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_pu', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_pu', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_pu', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_pu', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['pu_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['pu_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_pu', 'Coefficient'] * NHTS['ALL_PU']
df["util_404"] = vehf12.loc['pu', 'Coefficient'] + vehf12.loc['age4', 'Coefficient'] + vehf12.loc['i025_pu', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_pu', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_pu', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_pu', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_pu', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_pu', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_pu', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_pu', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_pu', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_pu', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_pu', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_pu', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_pu', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_pu', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_pu', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_pu', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_pu', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_pu', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_pu', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_pu', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_pu', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_pu', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_pu', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_pu', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['pu_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['pu_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_pu', 'Coefficient'] * NHTS['ALL_PU']
df["util_405"] = vehf12.loc['pu', 'Coefficient'] + vehf12.loc['age5', 'Coefficient'] + vehf12.loc['i025_pu', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_pu', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_pu', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_pu', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_pu', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_pu', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_pu', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_pu', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_pu', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_pu', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_pu', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_pu', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_pu', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_pu', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_pu', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_pu', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_pu', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_pu', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_pu', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_pu', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_pu', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_pu', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_pu', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_pu', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['pu_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['pu_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_pu', 'Coefficient'] * NHTS['ALL_PU']
df["util_406"] = vehf12.loc['pu', 'Coefficient'] + vehf12.loc['age6', 'Coefficient'] + vehf12.loc['i025_pu', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_pu', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_pu', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_pu', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_pu', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_pu', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_pu', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_pu', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_pu', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_pu', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_pu', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_pu', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_pu', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_pu', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_pu', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_pu', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_pu', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_pu', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_pu', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_pu', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_pu', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_pu', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_pu', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_pu', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['pu_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['pu_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_pu', 'Coefficient'] * NHTS['ALL_PU']
df["util_407"] = vehf12.loc['pu', 'Coefficient'] + vehf12.loc['age7', 'Coefficient'] + vehf12.loc['i025_pu', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_pu', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_pu', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_pu', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_pu', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_pu', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_pu', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_pu', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_pu', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_pu', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_pu', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_pu', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_pu', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_pu', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_pu', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_pu', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_pu', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_pu', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_pu', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_pu', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_pu', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_pu', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_pu', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_pu', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['pu_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['pu_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_pu', 'Coefficient'] * NHTS['ALL_PU']
df["util_408"] = vehf12.loc['pu', 'Coefficient'] + vehf12.loc['age8', 'Coefficient'] + vehf12.loc['i025_pu', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_pu', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_pu', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_pu', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_pu', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_pu', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_pu', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_pu', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_pu', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_pu', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_pu', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_pu', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_pu', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_pu', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_pu', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_pu', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_pu', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_pu', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_pu', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_pu', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_pu', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_pu', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_pu', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_pu', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['pu_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['pu_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_pu', 'Coefficient'] * NHTS['ALL_PU']
df["util_409"] = vehf12.loc['pu', 'Coefficient'] + vehf12.loc['age9', 'Coefficient'] + vehf12.loc['i025_pu', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_pu', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_pu', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_pu', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_pu', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_pu', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_pu', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_pu', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_pu', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_pu', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_pu', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_pu', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_pu', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_pu', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_pu', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_pu', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_pu', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_pu', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_pu', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_pu', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_pu', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_pu', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_pu', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_pu', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['pu_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['pu_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_pu', 'Coefficient'] * NHTS['ALL_PU']
df["util_410"] = vehf12.loc['pu', 'Coefficient'] + vehf12.loc['age10', 'Coefficient'] + vehf12.loc['i025_pu', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_pu', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_pu', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_pu', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_pu', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_pu', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_pu', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_pu', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_pu', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_pu', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_pu', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_pu', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_pu', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_pu', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_pu', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_pu', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_pu', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_pu', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_pu', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_pu', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_pu', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_pu', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_pu', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_pu', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['pu_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['pu_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_pu', 'Coefficient'] * NHTS['ALL_PU']
df["util_411"] = vehf12.loc['pu', 'Coefficient'] + vehf12.loc['age11', 'Coefficient'] + vehf12.loc['i025_pu', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_pu', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_pu', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_pu', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_pu', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_pu', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_pu', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_pu', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_pu', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_pu', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_pu', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_pu', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_pu', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_pu', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_pu', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_pu', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_pu', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_pu', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_pu', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_pu', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_pu', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_pu', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_pu', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_pu', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['pu_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['pu_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_pu', 'Coefficient'] * NHTS['ALL_PU']
df["util_412"] = vehf12.loc['pu', 'Coefficient'] + vehf12.loc['age12', 'Coefficient'] + vehf12.loc['i025_pu', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_pu', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_pu', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_pu', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_pu', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_pu', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_pu', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_pu', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_pu', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_pu', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_pu', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_pu', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_pu', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_pu', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_pu', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_pu', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_pu', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_pu', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_pu', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_pu', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_pu', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_pu', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_pu', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_pu', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['pu_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['pu_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_pu', 'Coefficient'] * NHTS['ALL_PU']
df["util_413"] = vehf12.loc['pu', 'Coefficient'] + vehf12.loc['age13', 'Coefficient'] + vehf12.loc['i025_pu', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_pu', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_pu', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_pu', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_pu', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_pu', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_pu', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_pu', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_pu', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_pu', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_pu', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_pu', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_pu', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_pu', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_pu', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_pu', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_pu', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_pu', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_pu', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_pu', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_pu', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_pu', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_pu', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_pu', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['pu_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['pu_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_pu', 'Coefficient'] * NHTS['ALL_PU']
df["util_414"] = vehf12.loc['pu', 'Coefficient'] + vehf12.loc['age14', 'Coefficient'] + vehf12.loc['i025_pu', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_pu', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_pu', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_pu', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_pu', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_pu', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_pu', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_pu', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_pu', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_pu', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_pu', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_pu', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_pu', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_pu', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_pu', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_pu', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_pu', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_pu', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_pu', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_pu', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_pu', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_pu', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_pu', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_pu', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['pu_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['pu_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_pu', 'Coefficient'] * NHTS['ALL_PU']
df["util_415"] = vehf12.loc['pu', 'Coefficient'] + vehf12.loc['age15', 'Coefficient'] + vehf12.loc['i025_pu', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_pu', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_pu', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_pu', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_pu', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_pu', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_pu', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_pu', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_pu', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_pu', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_pu', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_pu', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_pu', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_pu', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_pu', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_pu', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_pu', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_pu', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_pu', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_pu', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_pu', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_pu', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_pu', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_pu', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['pu_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['pu_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_pu', 'Coefficient'] * NHTS['ALL_PU']
df["util_416"] = vehf12.loc['pu', 'Coefficient'] + vehf12.loc['age16', 'Coefficient'] + vehf12.loc['i025_pu', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_pu', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_pu', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_pu', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_pu', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_pu', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_pu', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_pu', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_pu', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_pu', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_pu', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_pu', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_pu', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_pu', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_pu', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_pu', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_pu', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_pu', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_pu', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_pu', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_pu', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_pu', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_pu', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_pu', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['pu_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['pu_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_pu', 'Coefficient'] * NHTS['ALL_PU']
df["util_417"] = vehf12.loc['pu', 'Coefficient'] + vehf12.loc['age17', 'Coefficient'] + vehf12.loc['i025_pu', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_pu', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_pu', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_pu', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_pu', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_pu', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_pu', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_pu', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_pu', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_pu', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_pu', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_pu', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_pu', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_pu', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_pu', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_pu', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_pu', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_pu', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_pu', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_pu', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_pu', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_pu', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_pu', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_pu', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['pu_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['pu_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_pu', 'Coefficient'] * NHTS['ALL_PU']
df["util_418"] = vehf12.loc['pu', 'Coefficient'] + vehf12.loc['age18', 'Coefficient'] + vehf12.loc['i025_pu', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_pu', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_pu', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_pu', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_pu', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_pu', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_pu', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_pu', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_pu', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_pu', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_pu', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_pu', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_pu', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_pu', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_pu', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_pu', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_pu', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_pu', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_pu', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_pu', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_pu', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_pu', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_pu', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_pu', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['pu_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['pu_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_pu', 'Coefficient'] * NHTS['ALL_PU']
df["util_419"] = vehf12.loc['pu', 'Coefficient'] + vehf12.loc['age19', 'Coefficient'] + vehf12.loc['i025_pu', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_pu', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_pu', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_pu', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_pu', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_pu', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_pu', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_pu', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_pu', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_pu', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_pu', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_pu', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_pu', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_pu', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_pu', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_pu', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_pu', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_pu', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_pu', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_pu', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_pu', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_pu', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_pu', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_pu', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['pu_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['pu_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_pu', 'Coefficient'] * NHTS['ALL_PU']
df["util_420"] = vehf12.loc['pu', 'Coefficient'] + vehf12.loc['age20', 'Coefficient'] + vehf12.loc['i025_pu', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_pu', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_pu', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_pu', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_pu', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_pu', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_pu', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_pu', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_pu', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_pu', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_pu', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_pu', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_pu', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_pu', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_pu', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_pu', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_pu', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_pu', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_pu', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_pu', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_pu', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_pu', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_pu', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_pu', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['pu_pu', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['pu_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_pu', 'Coefficient'] * NHTS['ALL_PU']


In [13]:
#motorcycle aged 1-20 years
df["util_501"] = vehf12.loc['mc', 'Coefficient'] + vehf12.loc['i025_mc', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_mc', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_mc', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_mc', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_mc', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_mc', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_mc', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_mc', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_mc', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_mc', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_mc', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_mc', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_mc', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_mc', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_mc', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_mc', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_mc', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_mc', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_mc', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_mc', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_mc', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_mc', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_mc', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_mc', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['pu_mc', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['mc_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_mc', 'Coefficient'] * NHTS['ALL_MC']
df["util_502"] = vehf12.loc['mc', 'Coefficient'] + vehf12.loc['age2', 'Coefficient'] + vehf12.loc['i025_mc', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_mc', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_mc', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_mc', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_mc', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_mc', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_mc', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_mc', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_mc', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_mc', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_mc', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_mc', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_mc', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_mc', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_mc', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_mc', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_mc', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_mc', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_mc', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_mc', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_mc', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_mc', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_mc', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_mc', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['pu_mc', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['mc_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_mc', 'Coefficient'] * NHTS['ALL_MC']
df["util_503"] = vehf12.loc['mc', 'Coefficient'] + vehf12.loc['age3', 'Coefficient'] + vehf12.loc['i025_mc', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_mc', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_mc', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_mc', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_mc', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_mc', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_mc', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_mc', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_mc', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_mc', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_mc', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_mc', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_mc', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_mc', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_mc', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_mc', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_mc', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_mc', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_mc', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_mc', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_mc', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_mc', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_mc', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_mc', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['pu_mc', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['mc_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_mc', 'Coefficient'] * NHTS['ALL_MC']
df["util_504"] = vehf12.loc['mc', 'Coefficient'] + vehf12.loc['age4', 'Coefficient'] + vehf12.loc['i025_mc', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_mc', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_mc', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_mc', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_mc', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_mc', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_mc', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_mc', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_mc', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_mc', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_mc', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_mc', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_mc', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_mc', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_mc', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_mc', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_mc', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_mc', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_mc', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_mc', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_mc', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_mc', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_mc', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_mc', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['pu_mc', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['mc_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_mc', 'Coefficient'] * NHTS['ALL_MC']
df["util_505"] = vehf12.loc['mc', 'Coefficient'] + vehf12.loc['age5', 'Coefficient'] + vehf12.loc['i025_mc', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_mc', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_mc', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_mc', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_mc', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_mc', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_mc', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_mc', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_mc', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_mc', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_mc', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_mc', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_mc', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_mc', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_mc', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_mc', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_mc', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_mc', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_mc', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_mc', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_mc', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_mc', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_mc', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_mc', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['pu_mc', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['mc_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_mc', 'Coefficient'] * NHTS['ALL_MC']
df["util_506"] = vehf12.loc['mc', 'Coefficient'] + vehf12.loc['age6', 'Coefficient'] + vehf12.loc['i025_mc', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_mc', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_mc', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_mc', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_mc', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_mc', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_mc', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_mc', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_mc', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_mc', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_mc', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_mc', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_mc', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_mc', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_mc', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_mc', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_mc', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_mc', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_mc', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_mc', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_mc', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_mc', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_mc', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_mc', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['pu_mc', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['mc_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_mc', 'Coefficient'] * NHTS['ALL_MC']
df["util_507"] = vehf12.loc['mc', 'Coefficient'] + vehf12.loc['age7', 'Coefficient'] + vehf12.loc['i025_mc', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_mc', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_mc', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_mc', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_mc', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_mc', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_mc', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_mc', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_mc', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_mc', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_mc', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_mc', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_mc', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_mc', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_mc', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_mc', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_mc', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_mc', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_mc', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_mc', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_mc', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_mc', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_mc', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_mc', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['pu_mc', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['mc_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_mc', 'Coefficient'] * NHTS['ALL_MC']
df["util_508"] = vehf12.loc['mc', 'Coefficient'] + vehf12.loc['age8', 'Coefficient'] + vehf12.loc['i025_mc', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_mc', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_mc', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_mc', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_mc', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_mc', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_mc', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_mc', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_mc', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_mc', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_mc', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_mc', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_mc', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_mc', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_mc', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_mc', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_mc', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_mc', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_mc', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_mc', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_mc', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_mc', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_mc', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_mc', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['pu_mc', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['mc_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_mc', 'Coefficient'] * NHTS['ALL_MC']
df["util_509"] = vehf12.loc['mc', 'Coefficient'] + vehf12.loc['age9', 'Coefficient'] + vehf12.loc['i025_mc', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_mc', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_mc', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_mc', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_mc', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_mc', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_mc', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_mc', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_mc', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_mc', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_mc', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_mc', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_mc', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_mc', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_mc', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_mc', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_mc', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_mc', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_mc', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_mc', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_mc', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_mc', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_mc', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_mc', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['pu_mc', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['mc_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_mc', 'Coefficient'] * NHTS['ALL_MC']
df["util_510"] = vehf12.loc['mc', 'Coefficient'] + vehf12.loc['age10', 'Coefficient'] + vehf12.loc['i025_mc', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_mc', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_mc', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_mc', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_mc', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_mc', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_mc', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_mc', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_mc', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_mc', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_mc', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_mc', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_mc', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_mc', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_mc', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_mc', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_mc', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_mc', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_mc', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_mc', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_mc', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_mc', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_mc', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_mc', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['pu_mc', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['mc_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_mc', 'Coefficient'] * NHTS['ALL_MC']
df["util_511"] = vehf12.loc['mc', 'Coefficient'] + vehf12.loc['age11', 'Coefficient'] + vehf12.loc['i025_mc', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_mc', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_mc', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_mc', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_mc', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_mc', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_mc', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_mc', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_mc', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_mc', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_mc', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_mc', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_mc', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_mc', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_mc', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_mc', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_mc', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_mc', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_mc', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_mc', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_mc', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_mc', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_mc', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_mc', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['pu_mc', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['mc_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_mc', 'Coefficient'] * NHTS['ALL_MC']
df["util_512"] = vehf12.loc['mc', 'Coefficient'] + vehf12.loc['age12', 'Coefficient'] + vehf12.loc['i025_mc', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_mc', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_mc', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_mc', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_mc', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_mc', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_mc', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_mc', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_mc', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_mc', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_mc', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_mc', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_mc', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_mc', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_mc', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_mc', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_mc', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_mc', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_mc', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_mc', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_mc', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_mc', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_mc', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_mc', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['pu_mc', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['mc_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_mc', 'Coefficient'] * NHTS['ALL_MC']
df["util_513"] = vehf12.loc['mc', 'Coefficient'] + vehf12.loc['age13', 'Coefficient'] + vehf12.loc['i025_mc', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_mc', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_mc', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_mc', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_mc', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_mc', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_mc', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_mc', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_mc', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_mc', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_mc', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_mc', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_mc', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_mc', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_mc', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_mc', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_mc', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_mc', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_mc', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_mc', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_mc', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_mc', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_mc', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_mc', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['pu_mc', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['mc_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_mc', 'Coefficient'] * NHTS['ALL_MC']
df["util_514"] = vehf12.loc['mc', 'Coefficient'] + vehf12.loc['age14', 'Coefficient'] + vehf12.loc['i025_mc', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_mc', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_mc', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_mc', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_mc', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_mc', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_mc', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_mc', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_mc', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_mc', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_mc', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_mc', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_mc', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_mc', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_mc', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_mc', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_mc', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_mc', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_mc', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_mc', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_mc', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_mc', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_mc', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_mc', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['pu_mc', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['mc_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_mc', 'Coefficient'] * NHTS['ALL_MC']
df["util_515"] = vehf12.loc['mc', 'Coefficient'] + vehf12.loc['age15', 'Coefficient'] + vehf12.loc['i025_mc', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_mc', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_mc', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_mc', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_mc', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_mc', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_mc', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_mc', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_mc', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_mc', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_mc', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_mc', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_mc', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_mc', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_mc', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_mc', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_mc', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_mc', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_mc', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_mc', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_mc', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_mc', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_mc', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_mc', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['pu_mc', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['mc_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_mc', 'Coefficient'] * NHTS['ALL_MC']
df["util_516"] = vehf12.loc['mc', 'Coefficient'] + vehf12.loc['age16', 'Coefficient'] + vehf12.loc['i025_mc', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_mc', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_mc', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_mc', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_mc', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_mc', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_mc', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_mc', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_mc', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_mc', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_mc', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_mc', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_mc', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_mc', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_mc', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_mc', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_mc', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_mc', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_mc', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_mc', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_mc', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_mc', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_mc', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_mc', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['pu_mc', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['mc_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_mc', 'Coefficient'] * NHTS['ALL_MC']
df["util_517"] = vehf12.loc['mc', 'Coefficient'] + vehf12.loc['age17', 'Coefficient'] + vehf12.loc['i025_mc', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_mc', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_mc', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_mc', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_mc', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_mc', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_mc', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_mc', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_mc', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_mc', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_mc', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_mc', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_mc', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_mc', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_mc', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_mc', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_mc', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_mc', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_mc', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_mc', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_mc', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_mc', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_mc', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_mc', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['pu_mc', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['mc_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_mc', 'Coefficient'] * NHTS['ALL_MC']
df["util_518"] = vehf12.loc['mc', 'Coefficient'] + vehf12.loc['age18', 'Coefficient'] + vehf12.loc['i025_mc', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_mc', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_mc', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_mc', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_mc', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_mc', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_mc', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_mc', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_mc', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_mc', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_mc', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_mc', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_mc', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_mc', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_mc', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_mc', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_mc', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_mc', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_mc', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_mc', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_mc', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_mc', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_mc', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_mc', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['pu_mc', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['mc_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_mc', 'Coefficient'] * NHTS['ALL_MC']
df["util_519"] = vehf12.loc['mc', 'Coefficient'] + vehf12.loc['age19', 'Coefficient'] + vehf12.loc['i025_mc', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_mc', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_mc', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_mc', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_mc', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_mc', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_mc', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_mc', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_mc', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_mc', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_mc', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_mc', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_mc', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_mc', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_mc', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_mc', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_mc', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_mc', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_mc', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_mc', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_mc', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_mc', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_mc', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_mc', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['pu_mc', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['mc_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_mc', 'Coefficient'] * NHTS['ALL_MC']
df["util_520"] = vehf12.loc['mc', 'Coefficient'] + vehf12.loc['age20', 'Coefficient'] + vehf12.loc['i025_mc', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_mc', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_mc', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_mc', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_mc', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['vhgtdr_mc', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_mc', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['den3_mc', "Coefficient"] * NHTS['DEN3'] + vehf12.loc['den4_mc', 'Coefficient'] * NHTS['DEN4'] + vehf12.loc['den5_mc', 'Coefficient'] * NHTS['DEN5'] + vehf12.loc['den6_mc', 'Coefficient'] * NHTS['DEN6'] + vehf12.loc['den7_mc', 'Coefficient'] * NHTS['DEN7'] + vehf12.loc['den8_mc', 'Coefficient'] * NHTS['DEN8'] + vehf12.loc['sfo_mc', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_mc', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_mc', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_mc', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_mc', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_mc', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_mc', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_mc', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_mc', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['i025_age', "Coefficient"] * NHTS['INC025'] + vehf12.loc['i2550_age', "Coefficient"] * NHTS['INC2550']+ vehf12.loc['i100_age', "Coefficient"] * NHTS['INC100150']+ vehf12.loc['i150p_age', "Coefficient"] * NHTS['INC150P'] + vehf12.loc['imiss_age', "Coefficient"] * NHTS['INC_MISS'] + vehf12.loc['oneveh_age', "Coefficient"] * NHTS['ONEVEH'] + vehf12.loc['vhgtdr_age', "Coefficient"] * NHTS['VHGTDR'] + vehf12.loc['nchld_age', "Coefficient"] * NHTS['NUM_CHILD3'] + vehf12.loc['dstwkt_age', "Coefficient"] * NHTS['DSTWK'] + vehf12.loc['sfo_age', "Coefficient"] * NHTS['SFO'] + vehf12.loc['san_age', "Coefficient"] * NHTS['SAN'] + vehf12.loc['atl_age', "Coefficient"] * NHTS['ATL'] + vehf12.loc['sea_age', "Coefficient"] * NHTS['SEA'] + vehf12.loc['det_age', "Coefficient"] * NHTS['DET'] + vehf12.loc['msp_age', "Coefficient"] * NHTS['MSP'] + vehf12.loc['dca_age', "Coefficient"] * NHTS['DCA'] + vehf12.loc['oreg_age', "Coefficient"] * NHTS['OREG'] + vehf12.loc['ohio_age', "Coefficient"] * NHTS['OHIO'] + vehf12.loc['van_mc', 'Coefficient'] * NHTS['OWN_VAN'] + vehf12.loc['suv_mc', 'Coefficient'] * NHTS['OWN_SUV'] + vehf12.loc['pu_mc', 'Coefficient'] * NHTS['OWN_PU'] + vehf12.loc['mc_mc', 'Coefficient'] * NHTS['OWN_MC'] + vehf12.loc['only_mc', 'Coefficient'] * NHTS['ALL_MC']


In [24]:
df = df.head(10)
df

,util_101,util_102,util_103,util_104,util_105,util_106,util_107,util_108,util_109,util_110,...,util_511,util_512,util_513,util_514,util_515,util_516,util_517,util_518,util_519,util_520
HOUSEID,,,,,,,,,,,,,,,,,,,,,
30000007,0.07320,0.15670,0.10410,0.06388,-0.071685,-0.287052,-0.333675,-0.530013,-0.28110,-0.316078,...,-3.209833,-3.284029,-3.431878,-3.591844,-3.770898,-3.926233,-4.105241,-4.403203,-4.705256,-3.087140
30000007,0.07320,0.15670,0.10410,0.06388,-0.071685,-0.287052,-0.333675,-0.530013,-0.28110,-0.316078,...,-1.862073,-1.936269,-2.084119,-2.244084,-2.423138,-2.578473,-2.757481,-3.055443,-3.357496,-1.739380
30000007,0.07320,0.15670,0.10410,0.06388,-0.071685,-0.287052,-0.333675,-0.530013,-0.28110,-0.316078,...,-1.862073,-1.936269,-2.084119,-2.244084,-2.423138,-2.578473,-2.757481,-3.055443,-3.357496,-1.739380
30000007,0.07320,0.15670,0.10410,0.06388,-0.071685,-0.287052,-0.333675,-0.530013,-0.28110,-0.316078,...,-3.209833,-3.284029,-3.431878,-3.591844,-3.770898,-3.926233,-4.105241,-4.403203,-4.705256,-3.087140
30000008,0.04873,0.13223,0.07963,0.03941,-0.096155,-0.311522,-0.358145,-0.554483,-0.30557,-0.340548,...,-0.973982,-1.048178,-1.196028,-1.355993,-1.535047,-1.690382,-1.869390,-2.167352,-2.469405,-0.851289
30000008,0.04873,0.13223,0.07963,0.03941,-0.096155,-0.311522,-0.358145,-0.554483,-0.30557,-0.340548,...,-3.301513,-3.375709,-3.523558,-3.683524,-3.862578,-4.017913,-4.196920,-4.494883,-4.796936,-3.178819
30000008,0.04873,0.13223,0.07963,0.03941,-0.096155,-0.311522,-0.358145,-0.554483,-0.30557,-0.340548,...,-0.973982,-1.048178,-1.196028,-1.355993,-1.535047,-1.690382,-1.869390,-2.167352,-2.469405,-0.851289
30000008,0.04873,0.13223,0.07963,0.03941,-0.096155,-0.311522,-0.358145,-0.554483,-0.30557,-0.340548,...,-2.321742,-2.395938,-2.543787,-2.703753,-2.882807,-3.038142,-3.217150,-3.515112,-3.817165,-2.199049
30000012,0.01130,0.09480,0.04220,0.00198,-0.133585,-0.348952,-0.395575,-0.591913,-0.34300,-0.377978,...,-3.204822,-3.279018,-3.426867,-3.586833,-3.765887,-3.921222,-4.100230,-4.398192,-4.700245,-3.082129


In [35]:
exp_utils = np.exp(df)
exp_utils

,util_101,util_102,util_103,util_104,util_105,util_106,util_107,util_108,util_109,util_110,...,util_511,util_512,util_513,util_514,util_515,util_516,util_517,util_518,util_519,util_520
HOUSEID,,,,,,,,,,,,,,,,,,,,,
30000007,1.075946,1.169645,1.109711,1.065964,0.930824,0.750473,0.716287,0.588597,0.754953,0.729003,...,0.040363,0.037477,0.032326,0.027547,0.023031,0.019718,0.016486,0.012238,0.009048,0.045632
30000007,1.075946,1.169645,1.109711,1.065964,0.930824,0.750473,0.716287,0.588597,0.754953,0.729003,...,0.155350,0.144241,0.124417,0.106025,0.088643,0.075890,0.063451,0.047102,0.034822,0.175629
30000007,1.075946,1.169645,1.109711,1.065964,0.930824,0.750473,0.716287,0.588597,0.754953,0.729003,...,0.155350,0.144241,0.124417,0.106025,0.088643,0.075890,0.063451,0.047102,0.034822,0.175629
30000007,1.075946,1.169645,1.109711,1.065964,0.930824,0.750473,0.716287,0.588597,0.754953,0.729003,...,0.040363,0.037477,0.032326,0.027547,0.023031,0.019718,0.016486,0.012238,0.009048,0.045632
30000008,1.049937,1.141371,1.082886,1.040197,0.908323,0.732332,0.698972,0.574369,0.736704,0.711381,...,0.377576,0.350576,0.302393,0.257691,0.215446,0.184449,0.154218,0.114480,0.084635,0.426864
30000008,1.049937,1.141371,1.082886,1.040197,0.908323,0.732332,0.698972,0.574369,0.736704,0.711381,...,0.036827,0.034194,0.029494,0.025134,0.021014,0.017990,0.015042,0.011166,0.008255,0.041635
30000008,1.049937,1.141371,1.082886,1.040197,0.908323,0.732332,0.698972,0.574369,0.736704,0.711381,...,0.377576,0.350576,0.302393,0.257691,0.215446,0.184449,0.154218,0.114480,0.084635,0.426864
30000008,1.049937,1.141371,1.082886,1.040197,0.908323,0.732332,0.698972,0.574369,0.736704,0.711381,...,0.098103,0.091087,0.078568,0.066954,0.055977,0.047924,0.040069,0.029744,0.021990,0.110909
30000012,1.011364,1.099439,1.043103,1.001982,0.874953,0.705427,0.673293,0.553268,0.709639,0.685246,...,0.040566,0.037665,0.032489,0.027686,0.023147,0.019817,0.016569,0.012300,0.009093,0.045862


In [36]:
sum_exp_utils = exp_utils.sum(axis=1)

In [37]:
sum_exp_utils

HOUSEID
30000007    27.464980
30000007    30.271268
30000007    30.271268
30000007    27.464980
30000008    32.775083
30000008    26.091796
30000008    32.775083
30000008    26.866195
30000012    20.302250
30000012    20.302250
dtype: float64

In [45]:
prob = pd.DataFrame()
for col in df.columns: 
    prob[col] = exp_utils[col] / sum_exp_utils

In [46]:
prob

,util_101,util_102,util_103,util_104,util_105,util_106,util_107,util_108,util_109,util_110,...,util_511,util_512,util_513,util_514,util_515,util_516,util_517,util_518,util_519,util_520
HOUSEID,,,,,,,,,,,,,,,,,,,,,
30000007,0.039175,0.042587,0.040405,0.038812,0.033891,0.027325,0.026080,0.021431,0.027488,0.026543,...,0.001470,0.001365,0.001177,0.001003,0.000839,0.000718,0.000600,0.000446,0.000329,0.001661
30000007,0.035543,0.038639,0.036659,0.035214,0.030749,0.024792,0.023662,0.019444,0.024940,0.024082,...,0.005132,0.004765,0.004110,0.003502,0.002928,0.002507,0.002096,0.001556,0.001150,0.005802
30000007,0.035543,0.038639,0.036659,0.035214,0.030749,0.024792,0.023662,0.019444,0.024940,0.024082,...,0.005132,0.004765,0.004110,0.003502,0.002928,0.002507,0.002096,0.001556,0.001150,0.005802
30000007,0.039175,0.042587,0.040405,0.038812,0.033891,0.027325,0.026080,0.021431,0.027488,0.026543,...,0.001470,0.001365,0.001177,0.001003,0.000839,0.000718,0.000600,0.000446,0.000329,0.001661
30000008,0.032035,0.034824,0.033040,0.031737,0.027714,0.022344,0.021326,0.017525,0.022478,0.021705,...,0.011520,0.010696,0.009226,0.007862,0.006573,0.005628,0.004705,0.003493,0.002582,0.013024
30000008,0.040240,0.043744,0.041503,0.039867,0.034813,0.028068,0.026789,0.022013,0.028235,0.027265,...,0.001411,0.001311,0.001130,0.000963,0.000805,0.000690,0.000576,0.000428,0.000316,0.001596
30000008,0.032035,0.034824,0.033040,0.031737,0.027714,0.022344,0.021326,0.017525,0.022478,0.021705,...,0.011520,0.010696,0.009226,0.007862,0.006573,0.005628,0.004705,0.003493,0.002582,0.013024
30000008,0.039080,0.042484,0.040307,0.038718,0.033809,0.027258,0.026017,0.021379,0.027421,0.026479,...,0.003652,0.003390,0.002924,0.002492,0.002084,0.001784,0.001491,0.001107,0.000819,0.004128
30000012,0.049815,0.054154,0.051379,0.049353,0.043096,0.034746,0.033163,0.027252,0.034954,0.033752,...,0.001998,0.001855,0.001600,0.001364,0.001140,0.000976,0.000816,0.000606,0.000448,0.002259


In [ ]:
####
for each household:
    util=0
    find the HHAMP equivalent:
        variable = HHAM
        for veh in veh_type:
            for age in car_age:
                temp = veh + str(age)
                print(f"Utility: {temp}")
                # fetch co-efficient corresponding to the temp value
                # coef = df.loc[temp,"coefficient"]
                # util +=variable*coef      
####